In [2]:
import numpy as np
import pandas as pd
import os
import re
import pickle
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
os.environ['VISIBLE_CUDA_DEVICES'] = ""
from sklearn.metrics import accuracy_score, roc_curve, recall_score, auc, roc_auc_score

!ls ./RAP/HRPInception/

thr3_attributes_list.txt
thr3_v1_epoch001_valloss0.672014.hdf5_results.file
thr3_v1_epoch002_valloss0.685694.hdf5_results.file
thr3_v1_epoch003_valloss0.763411.hdf5_results.file
thr3_v1_epoch005_valloss0.770365.hdf5_results.file
thr3_v1_epoch006_valloss0.784905.hdf5_results.file
thr3_v1_epoch009_valloss0.792341.hdf5_results.file
thr3_v1_epoch011_valloss0.801338.hdf5_results.file
thr3_v1_epoch012_valloss0.808464.hdf5_results.file
thr3_v1_epoch013_valloss0.808747.hdf5_results.file
thr3_v1_epoch014_valloss0.821733.hdf5_results.file
thr3_v1_epoch015_valloss0.823892.hdf5_results.file
thr3_v1_epoch016_valloss0.826095.hdf5_results.file
thr3_v1_epoch017_valloss0.834860.hdf5_results.file
thr3_v1_epoch019_valloss0.835795.hdf5_results.file
thr3_v1_epoch020_valloss0.835968.hdf5_results.file
thr3_v1_epoch021_valloss0.839816.hdf5_results.file
thr3_v1_epoch022_valloss0.846303.hdf5_results.file
thr3_v1_epoch025_valloss0.847238.hdf5_results.file
thr3_v1_epoch026_valloss0.850762.hdf5_results.file
thr3_v

In [3]:
import scipy.io as sio

data = sio.loadmat('/home/anhaoran/data/pedestrian_attributes_RAP/RAP_annotation/RAP_annotation.mat')['RAP_annotation']
attributes_list = []
for i in range(data['attribute_eng'][0][0].shape[0]):
    attributes_list.append(data['attribute_eng'][0][0][i][0][0])
print(attributes_list)
attributes_list_chinese = []
for i in range(data['attribute_chinese'][0][0].shape[0]):
    attributes_list_chinese.append(data['attribute_chinese'][0][0][i][0][0])
print(attributes_list_chinese)

['Female', 'AgeLess16', 'Age17-30', 'Age31-45', 'BodyFat', 'BodyNormal', 'BodyThin', 'Customer', 'Clerk', 'hs-BaldHead', 'hs-LongHair', 'hs-BlackHair', 'hs-Hat', 'hs-Glasses', 'hs-Muffler', 'ub-Shirt', 'ub-Sweater', 'ub-Vest', 'ub-TShirt', 'ub-Cotton', 'ub-Jacket', 'ub-SuitUp', 'ub-Tight', 'ub-ShortSleeve', 'lb-LongTrousers', 'lb-Skirt', 'lb-ShortSkirt', 'lb-Dress', 'lb-Jeans', 'lb-TightTrousers', 'shoes-Leather', 'shoes-Sport', 'shoes-Boots', 'shoes-Cloth', 'shoes-Casual', 'attach-Backpack', 'attach-SingleShoulderBag', 'attach-HandBag', 'attach-Box', 'attach-PlasticBag', 'attach-PaperBag', 'attach-HandTrunk', 'attach-Other', 'action-Calling', 'action-Talking', 'action-Gathering', 'action-Holding', 'action-Pusing', 'action-Pulling', 'action-CarrybyArm', 'action-CarrybyHand', 'faceFront', 'faceBack', 'faceLeft', 'faceRight', 'occlusionLeft', 'occlusionRight', 'occlusionUp', 'occlusionDown', 'occlusion-Environment', 'occlusion-Attachment', 'occlusion-Person', 'occlusion-Other', 'up-Black

In [4]:
low = [11]
hs = [9,10,12,13,14]
ub = [15,16,17,18,19,20,21,22,23]
lb = [24,25,26,27,28,29]
sh = [30,31,32,33,34]
at = [35,36,37,38,39,40,41,42]
high = [0,1,2,3,4,5,6,7,8,43,44,45,46,47,48,49,50]
parts = [[len(low)], [len(hs), len(ub), len(lb), len(sh), len(at)], [len(high)]]
idx_indices = list(np.hstack((low, hs, ub, lb, sh, at, high)))
attributes_list_hiar = list(np.array(attributes_list)[idx_indices])
print("The total attributes labeled: ", len(attributes_list_hiar))
print(attributes_list_hiar)

The total attributes labeled:  51
['hs-BlackHair', 'hs-BaldHead', 'hs-LongHair', 'hs-Hat', 'hs-Glasses', 'hs-Muffler', 'ub-Shirt', 'ub-Sweater', 'ub-Vest', 'ub-TShirt', 'ub-Cotton', 'ub-Jacket', 'ub-SuitUp', 'ub-Tight', 'ub-ShortSleeve', 'lb-LongTrousers', 'lb-Skirt', 'lb-ShortSkirt', 'lb-Dress', 'lb-Jeans', 'lb-TightTrousers', 'shoes-Leather', 'shoes-Sport', 'shoes-Boots', 'shoes-Cloth', 'shoes-Casual', 'attach-Backpack', 'attach-SingleShoulderBag', 'attach-HandBag', 'attach-Box', 'attach-PlasticBag', 'attach-PaperBag', 'attach-HandTrunk', 'attach-Other', 'Female', 'AgeLess16', 'Age17-30', 'Age31-45', 'BodyFat', 'BodyNormal', 'BodyThin', 'Customer', 'Clerk', 'action-Calling', 'action-Talking', 'action-Gathering', 'action-Holding', 'action-Pusing', 'action-Pulling', 'action-CarrybyArm', 'action-CarrybyHand']


In [5]:
filename = r"../../pedestrian_attibutes_wpal/results/RAP_labels_pd.csv"
data = np.array(pd.read_csv(filename))[:, 1:]
length = len(data)
data_y = np.zeros((length, 51))
for i in range(length):
    data_y[i] = np.array(data[i, 1:52], dtype="float32")
data_y_hiar = data_y[:, idx_indices]
split = np.load('../../pedestrian_attibutes_wpal//results/RAP_partion.npy').item()
y_test = data_y[list(split['test'][0])]
y_test_hiar = data_y_hiar[list(split['test'][0])]
print("The shape of the y_test is: ", y_test.shape)
print("The shape of the y_test is: ", y_test_hiar.shape)

The shape of the y_test is:  (8317, 51)
The shape of the y_test is:  (8317, 51)


In [6]:
def mA(y_pred, y_true):
    M = len(y_pred)
    L = len(y_pred[0])
    res = 0
    for i in range(L):
        P = sum(y_true[:, i])
        N = M - P
        TP = sum(y_pred[:, i]*y_true[:, i])
        TN = list(y_pred[:, i]+y_true[:, i] == 0.).count(True)
        #print(TP, P, TN, N)
        #print(P,',', N,',', TP,',', TN)
        #if P != 0:
        res += TP/P + TN/N
        """else:
            res += 1 + TN/N"""
    return res / (2*L)

def acc(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        #print(np.shape(y_pred[i]*y_true[i]))
        if sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / (sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]))
            #print(y_pred[i])
            #print(y_true[i])
            #print(sum(y_pred[i]*y_true[i]) / (sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i])))
            M_ += 1
    #print(M_)
    return res / M_

def prec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_pred[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_pred[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

def rec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_true[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

In [7]:
def calculate_accuracy(gt_result, pt_result):
    ''' obtain the label-based and instance-based accuracy '''
    # compute the label-based accuracy
    if gt_result.shape != pt_result.shape:
        print('Shape beteen groundtruth and predicted results are different')
    # compute the label-based accuracy
    result = {}
    gt_pos = np.sum((gt_result == 1).astype(float), axis=0)
    gt_neg = np.sum((gt_result == -1).astype(float), axis=0)
    pt_pos = np.sum((gt_result == 1).astype(float) * (pt_result == 1).astype(float), axis=0)
    pt_neg = np.sum((gt_result == -1).astype(float) * (pt_result == -1).astype(float), axis=0)
    label_pos_acc = 1.0*pt_pos/gt_pos
    label_neg_acc = 1.0*pt_neg/gt_neg
    label_acc = (label_pos_acc + label_neg_acc)/2
    result['label_pos_acc'] = label_pos_acc
    result['label_neg_acc'] = label_neg_acc
    result['label_acc'] = label_acc
    # compute the instance-based accuracy
    # precision
    gt_pos = np.sum((gt_result == 1).astype(float), axis=1)
    pt_pos = np.sum((pt_result == 1).astype(float), axis=1)
    floatersect_pos = np.sum((gt_result == 1).astype(float)*(pt_result == 1).astype(float), axis=1)
    union_pos = np.sum(((gt_result == 1)+(pt_result == 1)).astype(float),axis=1)
    # avoid empty label in predicted results
    cnt_eff = float(gt_result.shape[0])
    for iter, key in enumerate(gt_pos):
        if key == 0:
            union_pos[iter] = 1
            pt_pos[iter] = 1
            gt_pos[iter] = 1
            cnt_eff = cnt_eff - 1
            continue
        if pt_pos[iter] == 0:
            pt_pos[iter] = 1
    instance_acc = np.sum(floatersect_pos/union_pos)/cnt_eff
    instance_precision = np.sum(floatersect_pos/pt_pos)/cnt_eff
    instance_recall = np.sum(floatersect_pos/gt_pos)/cnt_eff
    floatance_F1 = 2*instance_precision*instance_recall/(instance_precision+instance_recall)
    result['instance_acc'] = instance_acc
    result['instance_precision'] = instance_precision
    result['instance_recall'] = instance_recall
    result['instance_F1'] = floatance_F1
    return result

In [8]:
import sys
import keras.backend as K

def keras_mA(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)), axis=-1)
    true_negatives = np.sum(np.round(np.clip((1 - y_true) * (1 - y_pred), 0, 1)), axis=-1)
    possible_negatives = np.sum(np.round(np.clip(1 - y_true, 0, 1)), axis=-1)
    mean_acc = (true_positives / (possible_positives + sys.float_info.epsilon) + true_negatives / (possible_negatives + sys.float_info.epsilon)) / 2
    return mean_acc

def keras_acc(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)), axis=-1)
    possible_positives += np.sum(np.round(np.clip(y_pred, 0, 1)), axis=-1)
    possible_positives -= true_positives
    mean_acc = true_positives / (possible_positives + sys.float_info.epsilon)
    return np.average(mean_acc)

def keras_prec(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = np.sum(np.round(np.clip(y_pred, 0, 1)), axis=-1)
    mean_acc = true_positives / (possible_positives + sys.float_info.epsilon)
    return np.average(mean_acc)

def keras_rec(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)), axis=-1)
    mean_acc = true_positives / (possible_positives + sys.float_info.epsilon)
    return np.average(mean_acc)


def keras_acc_instance(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)), axis=-1)
    possible_positives += np.sum(np.round(np.clip(y_pred, 0, 1)), axis=-1)
    possible_positives -= true_positives
    mean_acc = true_positives / (possible_positives + sys.float_info.epsilon)
    return mean_acc

def keras_prec_instance(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = np.sum(np.round(np.clip(y_pred, 0, 1)), axis=-1)
    mean_acc = true_positives / (possible_positives + sys.float_info.epsilon)
    return mean_acc

def keras_rec_instance(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)), axis=-1)
    mean_acc = true_positives / (possible_positives + sys.float_info.epsilon)
    return mean_acc

def keras_K_mA(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=-1)
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)), axis=-1)
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)), axis=-1)
    mean_acc = (true_positives / (possible_positives + K.epsilon()) + true_negatives / (possible_negatives + K.epsilon())) / 2
    return mean_acc

Using TensorFlow backend.


In [9]:
def mA_thr(y_pred, y_true):
    M = len(y_pred)
    res = 0
    P = sum(y_true[:])
    N = M - P
    TP = sum(y_pred[:]*y_true[:])
    TN = list(y_pred[:]+y_true[:] == 0).count(True)
    if P != 0:
        res += TP/P + TN/N
    else:
        res += TN/N
    return res / 2


def find_best_fixed_threshold(preds, targs, do_plot=False):
    score = []
    thrs = np.arange(0, 1, 0.001)
    for thr in thrs:
        ### thr1/2
        #score.append(my_f2(targs, (preds > thr).astype(int) ))
        ### thr3
        score.append(mA_thr((preds[:, 0] > thr).astype(int), targs[:, 0]) )
        #res = calculate_accuracy(np.sign((preds[:] - thr)), np.sign(targs[:] - 0.5)) 
        #score.append(np.sum(res['label_acc'])/len(res['label_acc']))
    score = np.array(score)
    pm = score.argmax()
    best_thr, best_score = thrs[pm], score[pm].item()
    if do_plot:
        print('thr={best_thr:.9f}'.format(best_thr=best_thr), 'F2={best_score:.3f}'.format(best_score=best_score))
        plt.plot(thrs, score)
        plt.vlines(x=best_thr, ymin=score.min(), ymax=score.max())
        plt.text(best_thr+0.03, best_score-0.01, '$F_{2}=${best_score:.3f}'.format(best_score=best_score), fontsize=14);
        plt.show()
    return best_thr, best_score

In [10]:
reg = "HRPInception_RAP_v*"
weights = [s for s in os.listdir("../results/predictions/") 
          if re.match(reg, s)]
print(weights)

for w in weights:
    predictions_prob = np.load(
            "../results/predictions/" + w)
    print(w)
    
    predictions = predictions_prob
    y_test_hiar = data_y_hiar[list(split['test'][0])]
    label = y_test_hiar
    results = keras_mA(label, predictions)
    #print(results)
    #print(results.shape)
    print(np.average(results))
    k_acc = keras_acc(label, predictions)
    k_prec = keras_prec(label, predictions)
    k_rec = keras_rec(label, predictions)
    print(k_acc)
    print(k_prec)
    print(k_rec)
    print(2*k_prec*k_rec/(k_prec + k_rec))
    
    predictions = np.sign(predictions_prob-0.5)#[:, selected]
    y_test_hiar = data_y_hiar[list(split['test'][0])]
    #"""
    best_thr = np.zeros((1, 51))
    for i in range(51):
        best_thr[0, i], _ = find_best_fixed_threshold(predictions_prob[:, i:i+1], y_test_hiar[:, i:i+1])
    predictions = np.sign(predictions_prob - best_thr)
    #"""
    label = np.sign(y_test_hiar-0.5)#[:, selected]
    results = calculate_accuracy(label, predictions)
    print(results)
    print(np.sum(results['label_acc'])/len(results['label_acc']))

['HRPInception_RAP_v1_epoch002_valloss0.685694.hdf5.npy', 'HRPInception_RAP_v1_epoch017_valloss0.834860.hdf5.npy', 'HRPInception_RAP_v2_epoch029_valloss0.849231.hdf5.npy', 'HRPInception_RAP_v2_epoch044_valloss0.856305.hdf5.npy', 'HRPInception_RAP_v1_epoch001_valloss0.672014.hdf5.npy', 'HRPInception_RAP_v2_epoch045_valloss0.858335.hdf5.npy', 'HRPInception_RAP_v2_epoch014_valloss0.819026.hdf5.npy', 'HRPInception_RAP_v1_epoch019_valloss0.835795.hdf5.npy', 'HRPInception_RAP_v1_epoch012_valloss0.808464.hdf5.npy', 'HRPInception_RAP_v2_epoch024_valloss0.840333.hdf5.npy', 'HRPInception_RAP_v1_epoch026_valloss0.850762.hdf5.npy', 'HRPInception_RAP_v1_epoch015_valloss0.823892.hdf5.npy', 'HRPInception_RAP_v1_epoch016_valloss0.826095.hdf5.npy', 'HRPInception_RAP_v1_epoch021_valloss0.839816.hdf5.npy', 'HRPInception_RAP_v2_epoch003_valloss0.697382.hdf5.npy', 'HRPInception_RAP_v1_epoch009_valloss0.792341.hdf5.npy', 'HRPInception_RAP_v2_epoch025_valloss0.847681.hdf5.npy', 'HRPInception_RAP_v2_epoch005_

{'label_pos_acc': array([0.87224557, 0.52777778, 0.94098361, 0.775     , 0.78956522,
       0.8625    , 0.79655544, 0.76578411, 0.83643892, 0.76173469,
       0.82748538, 0.77403294, 0.82926829, 0.68953069, 0.78823529,
       0.8595433 , 0.91198502, 0.92460317, 0.92418773, 0.89673433,
       0.92555147, 0.83570557, 0.81455806, 0.94383149, 0.74509804,
       0.79865772, 0.8828125 , 0.81898067, 0.83490566, 0.77936963,
       0.76415094, 0.65891473, 0.80465116, 0.7018837 , 0.91133201,
       0.79012346, 0.76380368, 0.5972043 , 0.53932584, 0.73952435,
       0.68346253, 0.93542669, 0.90837696, 0.8115942 , 0.65185185,
       0.64537158, 0.70646766, 0.86666667, 0.71710526, 0.65363128,
       0.78256881]), 'label_neg_acc': array([0.8111588 , 0.9811617 , 0.89355124, 0.95120755, 0.85068458,
       0.75464368, 0.8467255 , 0.67048398, 0.88307378, 0.76624194,
       0.80184199, 0.78548896, 0.90322978, 0.81368159, 0.94942924,
       0.86911565, 0.9064628 , 0.87761934, 0.91206468, 0.90299124,
      

{'label_pos_acc': array([0.84893644, 0.83333333, 0.93442623, 0.7875    , 0.72521739,
       0.875     , 0.79870829, 0.74032587, 0.80331263, 0.79387755,
       0.80994152, 0.84680199, 0.83902439, 0.80505415, 0.9       ,
       0.86212839, 0.9588015 , 0.95238095, 0.96028881, 0.89143866,
       0.95036765, 0.84790565, 0.82668977, 0.94082247, 0.82352941,
       0.85458613, 0.90625   , 0.75219684, 0.91509434, 0.75931232,
       0.75471698, 0.72868217, 0.82790698, 0.66953317, 0.87435388,
       0.85185185, 0.72587842, 0.64645161, 0.59309791, 0.50849377,
       0.77002584, 0.93175487, 0.87958115, 0.76449275, 0.73703704,
       0.64928292, 0.74129353, 0.86666667, 0.76973684, 0.78212291,
       0.73302752]), 'label_neg_acc': array([0.81974249, 0.86149016, 0.87028857, 0.93416697, 0.86566779,
       0.76095666, 0.82923053, 0.666803  , 0.92226194, 0.70237533,
       0.82659375, 0.70697511, 0.88720414, 0.75858209, 0.85786179,
       0.8644898 , 0.8909161 , 0.87811531, 0.8931592 , 0.91555115,
      

{'label_pos_acc': array([0.85339447, 0.91666667, 0.91081967, 0.825     , 0.76173913,
       0.8625    , 0.78148547, 0.72810591, 0.79296066, 0.75612245,
       0.81871345, 0.80199157, 0.90243902, 0.80866426, 0.83529412,
       0.82722964, 0.96254682, 0.96825397, 0.94223827, 0.88437776,
       0.93106618, 0.83326556, 0.83015598, 0.94984955, 0.81372549,
       0.86651752, 0.8515625 , 0.82601054, 0.89150943, 0.79369628,
       0.74056604, 0.85271318, 0.86976744, 0.69328419, 0.86878728,
       0.88888889, 0.77300613, 0.61483871, 0.72953451, 0.56220676,
       0.7118863 , 0.95505191, 0.77486911, 0.8442029 , 0.65185185,
       0.62059974, 0.72636816, 0.61111111, 0.76315789, 0.59217877,
       0.80733945]), 'label_neg_acc': array([0.83690987, 0.94843618, 0.88103651, 0.88966532, 0.78158099,
       0.76071385, 0.83867472, 0.6338105 , 0.91230534, 0.71936448,
       0.81033242, 0.7169646 , 0.85318047, 0.69800995, 0.9311403 ,
       0.88789116, 0.87498394, 0.85765654, 0.88930348, 0.91323748,
      

{'label_pos_acc': array([0.87186346, 0.91666667, 0.89901639, 0.8375    , 0.74782609,
       0.8       , 0.75941873, 0.83197556, 0.76190476, 0.7994898 ,
       0.83333333, 0.77556492, 0.85365854, 0.84115523, 0.82941176,
       0.83498492, 0.94756554, 0.93253968, 0.93501805, 0.90335393,
       0.93106618, 0.84099227, 0.80372617, 0.94984955, 0.88235294,
       0.83146905, 0.875     , 0.75043937, 0.8254717 , 0.74498567,
       0.83962264, 0.84496124, 0.86976744, 0.59131859, 0.88071571,
       0.91358025, 0.75962075, 0.64150538, 0.67094703, 0.77560265,
       0.71447028, 0.93681945, 0.89528796, 0.75362319, 0.74074074,
       0.62190352, 0.80597015, 0.86666667, 0.80263158, 0.62569832,
       0.77522936]), 'label_neg_acc': array([0.81974249, 0.9408284 , 0.87293875, 0.91369376, 0.75794368,
       0.77127595, 0.83232699, 0.48220859, 0.91370947, 0.65895863,
       0.77392431, 0.72660358, 0.85626233, 0.6460199 , 0.89137106,
       0.88      , 0.87177181, 0.8549287 , 0.89154229, 0.90679227,
      

...

{'label_pos_acc': array([0.79645905, 0.91666667, 0.91409836, 0.8375    , 0.70086957,
       0.75      , 0.79763186, 0.81262729, 0.8447205 , 0.76632653,
       0.87280702, 0.76407507, 0.86341463, 0.77978339, 0.90588235,
       0.84618699, 0.9494382 , 0.96825397, 0.94223827, 0.88305384,
       0.92095588, 0.87149248, 0.78596187, 0.9558676 , 0.79411765,
       0.82550336, 0.921875  , 0.7486819 , 0.83962264, 0.73638968,
       0.84433962, 0.78294574, 0.86046512, 0.65561016, 0.88548708,
       0.82716049, 0.76770775, 0.6688172 , 0.65810594, 0.75861511,
       0.6498708 , 0.95530514, 0.85078534, 0.7826087 , 0.6037037 ,
       0.65449804, 0.73631841, 0.88888889, 0.76973684, 0.58659218,
       0.78623853]), 'label_neg_acc': array([0.85622318, 0.93394518, 0.87470554, 0.88402599, 0.83363472,
       0.85953624, 0.83372039, 0.55446489, 0.86545826, 0.72722983,
       0.74931645, 0.75902559, 0.86279586, 0.74639303, 0.87430956,
       0.87210884, 0.87691122, 0.83223807, 0.90273632, 0.91803008,
      

{'label_pos_acc': array([0.87874156, 0.91666667, 0.85442623, 0.74375   , 0.74086957,
       0.7       , 0.69429494, 0.72606925, 0.70600414, 0.72397959,
       0.80190058, 0.68364611, 0.85365854, 0.72563177, 0.81764706,
       0.81602757, 0.8988764 , 0.88095238, 0.89530686, 0.82480141,
       0.89613971, 0.80235868, 0.79722704, 0.87261785, 0.84313725,
       0.7733035 , 0.7890625 , 0.76449912, 0.83018868, 0.69340974,
       0.60849057, 0.72093023, 0.84651163, 0.63513514, 0.86441352,
       0.75308642, 0.66871166, 0.6027957 , 0.69582665, 0.40883352,
       0.62661499, 0.91858698, 0.87172775, 0.77173913, 0.7       ,
       0.61147327, 0.57711443, 0.73333333, 0.76315789, 0.65363128,
       0.62752294]), 'label_neg_acc': array([0.69313305, 0.94336433, 0.83583628, 0.78300846, 0.58757427,
       0.83234187, 0.7970274 , 0.50388548, 0.79640031, 0.63520529,
       0.71434739, 0.71591307, 0.79598126, 0.6141791 , 0.76310298,
       0.80517007, 0.84440447, 0.84488531, 0.82512438, 0.90943646,
      

{'label_pos_acc': array([0.84307731, 0.83333333, 0.92      , 0.8125    , 0.74782609,
       0.7875    , 0.79332616, 0.84215886, 0.86335404, 0.78673469,
       0.87353801, 0.80888548, 0.82926829, 0.79422383, 0.85294118,
       0.84295562, 0.94007491, 0.95634921, 0.93140794, 0.88172992,
       0.94761029, 0.85685238, 0.85008666, 0.94784353, 0.84313725,
       0.87546607, 0.859375  , 0.75746924, 0.8490566 , 0.71633238,
       0.76415094, 0.82945736, 0.83255814, 0.7035217 , 0.89184891,
       0.86419753, 0.74233129, 0.6344086 , 0.62359551, 0.83222779,
       0.74289406, 0.96062294, 0.85340314, 0.7826087 , 0.62962963,
       0.53846154, 0.74129353, 0.87777778, 0.75657895, 0.7150838 ,
       0.77431193]), 'label_neg_acc': array([0.83476395, 0.97536529, 0.89767373, 0.90266029, 0.8840093 ,
       0.84945975, 0.85431181, 0.57041581, 0.87758489, 0.7266006 ,
       0.78212692, 0.74605678, 0.9193787 , 0.75273632, 0.91788388,
       0.88789116, 0.8883464 , 0.8633602 , 0.89514925, 0.92282267,
      

{'label_pos_acc': array([0.86039995, 0.80555556, 0.91147541, 0.85625   , 0.70086957,
       0.825     , 0.77125942, 0.83910387, 0.82194617, 0.81071429,
       0.81578947, 0.77824588, 0.86829268, 0.81949458, 0.88823529,
       0.82722964, 0.93258427, 0.93253968, 0.93140794, 0.88923213,
       0.92095588, 0.81333876, 0.82279029, 0.93981946, 0.7745098 ,
       0.82699478, 0.8984375 , 0.83304042, 0.85377358, 0.67048711,
       0.78301887, 0.84496124, 0.8744186 , 0.63677314, 0.87475149,
       0.88888889, 0.71221417, 0.67397849, 0.61155698, 0.69373888,
       0.67312661, 0.94492277, 0.87696335, 0.78623188, 0.77037037,
       0.66362451, 0.78109453, 0.84444444, 0.78947368, 0.68715084,
       0.7733945 ]), 'label_neg_acc': array([0.79399142, 0.91836735, 0.87190813, 0.86171387, 0.82459313,
       0.78426612, 0.8365072 , 0.50947512, 0.85371458, 0.63835142,
       0.78011225, 0.72397476, 0.83678501, 0.66044776, 0.85135633,
       0.85959184, 0.90016703, 0.87439554, 0.89042289, 0.90332176,
      

{'label_pos_acc': array([0.87886893, 0.66666667, 0.93770492, 0.85625   , 0.77913043,
       0.6625    , 0.77125942, 0.7311609 , 0.83436853, 0.80561224,
       0.78947368, 0.82420529, 0.8097561 , 0.75090253, 0.83529412,
       0.86945282, 0.91385768, 0.9484127 , 0.92057762, 0.9086496 ,
       0.94669118, 0.80764538, 0.83102253, 0.93681043, 0.82352941,
       0.82102908, 0.8984375 , 0.82776801, 0.74056604, 0.70773639,
       0.83962264, 0.60465116, 0.84186047, 0.64946765, 0.91570577,
       0.80246914, 0.71388734, 0.61892473, 0.53772071, 0.72075716,
       0.71705426, 0.96543429, 0.90837696, 0.80072464, 0.68888889,
       0.58148631, 0.56716418, 0.86666667, 0.73026316, 0.55865922,
       0.76330275]), 'label_neg_acc': array([0.81974249, 0.98393914, 0.89355124, 0.92681133, 0.87522604,
       0.89474323, 0.8841926 , 0.69475119, 0.88511616, 0.71433066,
       0.85278457, 0.7276551 , 0.92578895, 0.75360697, 0.86424451,
       0.8552381 , 0.91519979, 0.83137012, 0.89863184, 0.88679557,
      

In [18]:
all_data = pd.DataFrame()
root = "./RAP"
for dirpath, dirnames, filenames in os.walk(root):
    for filepath in filenames:
        if filepath.endswith(".txt"):
            continue
        file = open(os.path.join(dirpath, filepath), "rb")
        data = pickle.load(file)
        if "label_pos_acc" in data.keys():
            data.pop("label_pos_acc")
            data.pop("label_neg_acc")
            data.pop("label_acc")
        #print(dirpath)
        if filepath.find("_valloss") != -1:
            data["model"] = dirpath[dirpath.rindex("/")+1:] + "_" + filepath[:filepath.index("_valloss")]
        else:
            data["model"] = dirpath[dirpath.rindex("/")+1:] + "_" + filepath[:filepath.index("_model")]
        """
        # all_data.update(data)
        for key, value in data.items():
            if key not in all_data.keys():
                all_data.setdefault(key, [])
            all_data[key].append(value)
            """
        if filepath.find("_valloss") != -1:
            data["ma"] = float(filepath[filepath.rindex("valloss")+7:filepath.rindex(".h")])
        else:
            data["ma"] = 0
        data = pd.DataFrame(data, index=[0])
        all_data = all_data.append(data)

        
order = ["model", "label_ma", "ma", "instance_acc", "instance_precision", "instance_recall", "instance_F1"]
all_data = all_data[order]
all_data.sort_values(by="ma", ascending=False)

,model,label_ma,ma,instance_acc,instance_precision,instance_recall,instance_F1
0,HRPInception_thr3_v1_epoch039,0.813953,0.861716,0.509800,0.559537,0.837118,0.670744
0,HRPInception_thr3_v2_epoch053,0.807129,0.859308,0.514665,0.565563,0.836469,0.674843
0,HRPInception_thr3_v2_epoch048,0.811006,0.858574,0.500279,0.551493,0.825732,0.661309
0,HRPInception_thr3_v2_epoch045,0.807543,0.858335,0.506730,0.557569,0.831254,0.667445
0,HRPInception_thr3_v1_epoch033,0.812681,0.857515,0.499193,0.544262,0.843271,0.661549
0,HRPInception_thr3_v2_epoch044,0.807114,0.856305,0.494336,0.541868,0.835498,0.657385
0,HRPInception_thr3_v2_epoch034,0.816495,0.856187,0.482789,0.528533,0.832330,0.646522
0,HRPInception_thr3_v2_epoch033,0.816046,0.855023,0.488340,0.536983,0.828351,0.651578
0,HRPInception_thr3_v1_epoch029,0.811222,0.851223,0.481754,0.531380,0.823156,0.645843
0,HRPInception_thr3_v1_epoch026,0.812478,0.850762,0.479051,0.529947,0.818314,0.643292


In [17]:
all_data.sort_values(by="ma", ascending=False)

,model,label_ma,ma,instance_acc,instance_precision,instance_recall,instance_F1
0,HRPInception_thr3_v1_epoch039,0.813953,0.861716,0.509800,0.559537,0.837118,0.670744
0,HRPInception_thr3_v2_epoch053,0.807129,0.859308,0.514665,0.565563,0.836469,0.674843
0,HRPInception_thr3_v2_epoch048,0.811006,0.858574,0.500279,0.551493,0.825732,0.661309
0,HRPInception_thr3_v2_epoch045,0.807543,0.858335,0.506730,0.557569,0.831254,0.667445
0,HRPInception_thr3_v1_epoch033,0.812681,0.857515,0.499193,0.544262,0.843271,0.661549
0,HRPInception_thr3_v2_epoch044,0.807114,0.856305,0.494336,0.541868,0.835498,0.657385
0,HRPInception_thr3_v2_epoch034,0.816495,0.856187,0.482789,0.528533,0.832330,0.646522
0,HRPInception_thr3_v2_epoch033,0.816046,0.855023,0.488340,0.536983,0.828351,0.651578
0,HRPInception_thr3_v1_epoch029,0.811222,0.851223,0.481754,0.531380,0.823156,0.645843
0,HRPInception_thr3_v1_epoch026,0.812478,0.850762,0.479051,0.529947,0.818314,0.643292


In [19]:
label_acc = pickle.load(open("./RAP/HRPInception/thr3_v1_epoch039_valloss0.861716.hdf5_results.file", "rb"))['label_acc']
i = 0
with open("./RAP/HRPInception/thr3_attributes_list.txt", "r") as f:
    while True:
        lines = f.readline()
        if not lines:
            break
        print(lines, label_acc[i])
        i += 1

hs-BlackHair
 0.8214547721702984
hs-BaldHead
 0.9045338056327067
hs-LongHair
 0.9143184846202861
hs-Hat
 0.8692913295329165
hs-Glasses
 0.82520840587198
hs-Muffler
 0.7766207357047469
ub-Shirt
 0.836644023217743
ub-Sweater
 0.7207912152903595
ub-Vest
 0.8676441439370033
ub-TShirt
 0.7616531109848376
ub-Cotton
 0.8251077293295598
ub-Jacket
 0.7826360286758964
ub-SuitUp
 0.8619344421994515
ub-Tight
 0.7624286060671372
ub-ShortSleeve
 0.8849262449548372
lb-LongTrousers
 0.8674817035819067
lb-Skirt
 0.9253599870263673
lb-ShortSkirt
 0.9037768527539141
lb-Dress
 0.9084089929414301
lb-Jeans
 0.9064569648718461
lb-TightTrousers
 0.9183220489572229
shoes-Leather
 0.841900337262064
shoes-Sport
 0.784166363106986
shoes-Boots
 0.9308513655721262
shoes-Cloth
 0.8803867864857446
shoes-Casual
 0.7353767300179929
attach-Backpack
 0.864056394248382
attach-SingleShoulderBag
 0.812585571150285
attach-HandBag
 0.8422567015469138
attach-Box
 0.7823532669359385
attach-PlasticBag
 0.8271571240673705
attach-